# Ecercise sheet 1

## Exercise 1: Theoretical questions



### Exercise 1.1

Explain collision resistance in your own words. Write your answer in the cell below.

For an hash function $H()$ being collision resistant means that for any pair of input $x$ and $y$ , with $x \neq y$ we'll have that $H(x) \neq H(y) $. Formally, collision may happen but we're interested in making the computationally infeasible. 

## Exercise 1.2

What’s the difference between puzzle-friendliness (p.f.) and hiding? Explain
in your own words. Write your answer in the cell below.

We can consider hiding as a special case of p.f.. \
In p.f. We want the $x$ of $H(k||x) = y$ being hard to find, but not infeasible since our goal is to allow for a puzzle solution. In the case of hiding, finding $x$ must be infeasible since our goal is to avoid the reconstruction of $x$ by means of its hash value $y$. 
So, hiding is just a special case of p.f where the output space $Y$ is simply the singleton ${y}$. 

## Exercise 2: A very simple block chain 

Some helpful links: 
1. https://www.geeksforgeeks.org/how-to-get-value-from-address-in-python/
2. https://datagy.io/python-sha256/

Below you can find some imports that might be useful. The 'hashlib' library includes various hashing algorithms such as hashlib.sha256. 'datetime' allows you to get the exact time and could be used to generate the timestamps. Finally 'json' is needed to encode the blocks (dictionaries), so that hashlib can use them as input to their hashing algorithms.

Hint: In CPython, you can use id(object) to get the pointer to a PyObject.

In [3]:
# imports
import hashlib
import datetime
import json
import ctypes

The three dictionaries below represent hypothetical transactions and serve as the inputs to the three blocks of our simple block chain that you are supposed to create. This means, that in each block that you create with the classes block() and simple_block_chain(), you should enclose one of the three transaction dictionaries.

In [1]:
transactions_1 = {
    't1': 'Alice sends 10 BTC to Bob',
    't2': 'Charlie sends 2.5 BTC to Bob',
    't3': 'Bob sends 5 BTC to Sarah'
}

transactions_2 = {
    't1': 'Bob sends 4.5 BTC to Tim',
    't2': 'Sarah sends 1 BTC to Mary',
    't3': 'Paul sends 2 BTC to Jessica'
}

transactions_3 = {
    't1': 'Jessica sends 1.5 BTC to Charlie',
    't2': 'Jessica sends 0.05 BTC to Carmen',
    't3': 'Steffan sends 3.12 BTC to Michel'
}

The Genesis block: The Genesis block is the first block in our simple block chain. Use it as the starting point to create new blocks. This means, that the first block that you create should include a hash pointer to the genesis block. The second block you create should include a hash pointer to the first block you created and so on.

Note, that no transactions are enclosed in our Genesis block. Theoretically this would be possible though.

In [4]:
genesis_block = {
            'timestamp': str(datetime.datetime.now()),
            'previous_block_hash': 'There is no previous block',
            'pointer' : 'There is no pointer to a previous block'
        }
genesis_block

{'timestamp': '2023-11-08 10:54:16.997192',
 'previous_block_hash': 'There is no previous block',
 'pointer': 'There is no pointer to a previous block'}

In the cell below you can find some sample code, that allows you to create a hash pointer to a block (here this block is the Genesis block). In our case, a hash pointer is a dictionary with two key:value pairs: 'previous_block_hash' (the hash of the previous block) and 'pointer' (the pointer to the previous block). You can use this hash pointer structure to get access to the hash and pointer of the previous block when implementing the classes block() and simple_block_chain().

In [5]:
#get the address in memory (the pointer) of the genesis block
genesis_id = id(genesis_block)

#get the hash of the genesis block
genesis_encoded = json.dumps(genesis_block, sort_keys=True).encode() 
genesis_hash = hashlib.sha256(genesis_encoded).hexdigest()

#hash pointer of genesis block
genesis_hash_pointer = {
    'previous_block_hash' : genesis_hash,
    'pointer' : genesis_id
}

genesis_hash_pointer

{'previous_block_hash': '6354db1ad8ecab4332fd982feb17a70ab0c47e1125e0e35a698a11207ec1fc7c',
 'pointer': 2804893317056}

### Exercise 2.1

In [6]:
class Block(object):
    def __init__(self):
        '''
        Upon instantiation, create an empty block of type dictionary.
        self.prev_block is set to None. This should be updated to the previous block (dictionary).
        self.prev_block_hash is set to None. This should be updated to the hash of the previous block.
        self.prev_block_pointer is set to None. This should be updated to the pointer to the previous 
        block.
        '''
        self.block = {}
        self.prev_block = None
        self.prev_block_hash = None
        self.prev_block_pointer = None
    
    def get_previous_block(self, pointer): 
        '''
        Use the pointer to the previous block (stored in the corresponding hash pointer) to get the 
        previous block from memory. Update self.prev_block.
        Hint: Use ctypes.cast, refer to the links provided above for a short introduction.
        pointer: Address in memory of the previous block.
        '''
        self.prev_block = ctypes.cast(pointer, ctypes.py_object).value
        self.prev_block_hash = self.block['previous_block_hash'] 
        self.prev_block_pointer = self.block['pointer']
        
     
    def verify_block_content(self, hash_of_prev_block):
        '''
        This method receives the hash of the previous block as input (we get it from the corresponding
        hash pointer) and verifies, whether or not the block that we store in self.prev block has 
        changed since we last visited the block. Compare the hashes, if they don't match, throw an 
        exception.
        hash_of_prev_block: Hash of the previous block
        '''
        
        if hash_of_prev_block != self.prev_block_hash:
            raise Exception ('Hash values does not correspond' )



    
    def create_new_block(self, hash_pointer: dict, transactions: dict):
        '''
        Create a new block, given a hash pointer and some transactions.
        hash_pointer: The hash pointer to the previous block of type dictionary
        transactions: Some transactions that should be enclosed in the new block (type dictionary)
        return: Return the new block
        '''
        
        new_block = Block()  # Create a new instance of the Block class

        # Set the attributes of the new block
        new_block.block = {
            'timestamp': str(datetime.datetime.now()),
            **hash_pointer,
            **transactions
        }
        new_block.prev_block=self
        return new_block

             

### Exercise 2.2

In [18]:
class simple_block_chain(object):
    def __init__(self):
        '''
        self.latest_hash_pointer: Set to the hash pointer of the genesis block, as this is the first
        block in our simple block chain. This should be updated whenever a new block is created.
        
        self.newest_block: Set to the genesis block. This should be updated when a new block is
        created.
        
        self.chain: Store the blocks of the block chain recursively in self.chain. This means, that 
        the first element is the head of the block chain and the last element is the genesis block.
        '''
        gen_block= Block() # converting genesis block to a Block class s.t. we can safely use gen_block.create_new_block()
        gen_block.block= genesis_block 
        
        self.latest_hash_pointer = genesis_hash_pointer
        self.newest_block = gen_block 
        self.chain = []
    
    def create_block(self, transactions: dict):
        '''
        Use the newly implemented class block() and its method create_new_block() to create a new 
        block.
        Update self.newest_block and self.latest_hash_pointer whenever a new block is created.
        transactions: Dictionary with transactions that should be enclosed in the next block
        '''
            
        new_block= self.newest_block.create_new_block(self.latest_hash_pointer, transactions)
        
        new_block_id = id(new_block)

        #get the hash of the genesis block
        new_block_encoded = json.dumps(new_block.block, sort_keys=True).encode() 
        new_block_hash = hashlib.sha256(new_block_encoded).hexdigest()

        #hash pointer of genesis block
        new_block_hash_pointer = {
            'previous_block_hash' : new_block_hash,
            'pointer' : new_block_id
        }
        
        self.latest_hash_pointer= new_block_hash_pointer
        self.newest_block = new_block
        
                
        
    def get_head_of_block_chain(self):
        '''
        Return the head of the block chain. The head of a block chain is the newest block.
        '''
        return self.newest_block 
        
    
    def get_chain_recursively(self, hash_pointer):
        '''
        This method receives a hash pointer as input and stores the corresponding block and its 
        predecessors (all blocks that were created before) in self.chain. When you reach the Genesis
        block, stop and return self.chain. 
        '''

        
        current_block = ctypes.cast(hash_pointer['pointer'], ctypes.py_object).value
        
        if type(current_block)==dict: #for some reason the genesis block becomes of type dict when i try to retrieve it, 
                                      #but when i create the blockchain it is from Block class
            return self.chain
       
        self.chain.append(current_block.block) 
        
        current_block.get_previous_block(current_block.block['pointer'])
        
        prev_hash_pointer = {
            'previous_block_hash': current_block.prev_block_hash,
            'pointer': current_block.prev_block_pointer
        }
    
        return(self.get_chain_recursively(prev_hash_pointer))

    
    
        

### Exercise 2.3 

Now, use your new classes block() and simple_block_chain() to build a block chain with four blocks in total: The Genesis block and three blocks in which transactions_1, transactions_2 and transactions_3 are enclosed. Use the Genesis block and its hash pointer as starting point.

In [22]:
#instanciate blockchain
blockchain=simple_block_chain()

In [23]:
# generate 3 blocks
blockchain.create_block(transactions_1)
blockchain.create_block(transactions_2)
blockchain.create_block(transactions_3)

In [26]:
blockchain.get_head_of_block_chain()

{'timestamp': '2023-11-08 10:58:38.646472',
 'previous_block_hash': '5560ae21d8b353c720fed61d57d8db2c50ba1a73dba455c473e0aec57cd3c6cc',
 'pointer': 2804876670480,
 't1': 'Jessica sends 1.5 BTC to Charlie',
 't2': 'Jessica sends 0.05 BTC to Carmen',
 't3': 'Steffan sends 3.12 BTC to Michel'}

In [27]:
chain = blockchain.get_chain_recursively(blockchain.latest_hash_pointer)

In [29]:
chain

[{'timestamp': '2023-11-08 10:58:38.646472',
  'previous_block_hash': '5560ae21d8b353c720fed61d57d8db2c50ba1a73dba455c473e0aec57cd3c6cc',
  'pointer': 2804876670480,
  't1': 'Jessica sends 1.5 BTC to Charlie',
  't2': 'Jessica sends 0.05 BTC to Carmen',
  't3': 'Steffan sends 3.12 BTC to Michel'},
 {'timestamp': '2023-11-08 10:58:38.646472',
  'previous_block_hash': '4e3c4e31f2d85327ff4b001ce6b06428214e061ec498c88da89f925e66796d74',
  'pointer': 2804876665968,
  't1': 'Bob sends 4.5 BTC to Tim',
  't2': 'Sarah sends 1 BTC to Mary',
  't3': 'Paul sends 2 BTC to Jessica'},
 {'timestamp': '2023-11-08 10:58:38.641342',
  'previous_block_hash': '6354db1ad8ecab4332fd982feb17a70ab0c47e1125e0e35a698a11207ec1fc7c',
  'pointer': 2804893317056,
  't1': 'Alice sends 10 BTC to Bob',
  't2': 'Charlie sends 2.5 BTC to Bob',
  't3': 'Bob sends 5 BTC to Sarah'}]

### Exercise 2.4

Change one of the transactions that we use as block input. What do you
observe or would expect to oberve (in case you were not able to implement both classes) if
you would create a new block with the updated transactions? In what way does the new
block differ to the block that was created with the original transactions?

If we create a new block with the updated transactions, the block will have his hash value and id which will serve as ab hash pointer for the next block, and it will also contain the hash pointer from the previous block along with its timestamp and corresponding transactions. So, the hash value and pointer of the two blocks will differ leading to an inconsistency in the blockchain. If the purpose of the transaction update is about manipulating the content of an already verified block, this will be immediately detected by the fact that the new block will store a wrong pointer and hash value. 